In [ ]:
# default_exp model

# The model

> from the paper from [Paletta et al](https://arxiv.org/pdf/2104.12419v1.pdf)

We will try to implement as close as possible the architecture from the paper `ECLIPSE : Envisioning Cloud Induced Perturbations in Solar Energy`

![Image](images/eclipse_diagram.png)

In [ ]:
#export
from eclipse_pytorch.imports import *
from eclipse_pytorch.layers import *

## 1. Spatial Downsampler
> A resnet encoder to get image features

You could use any spatial downsampler as you want, but the paper states a simple resnet arch...

In [ ]:
#export
class SpatialDownsampler(nn.Module):
    
    def __init__(self, in_channels=3):
        super().__init__()
        self.conv1 = ConvBlock(in_channels, 64, kernel_size=7, stride=1)
        self.blocks = nn.Sequential(ResBlock(64, 64, kernel_size=3, stride=2), 
                                    ResBlock(64, 128, kernel_size=3, stride=2), 
                                    ResBlock(128,256, kernel_size=3, stride=2))
    
    def forward(self, x):
        return self.blocks(self.conv1(x))

In [ ]:
sd = SpatialDownsampler()

In [ ]:
images = [torch.rand(1, 3, 64, 64) for _ in range(4)]
features = torch.stack([sd(image) for image in images], dim=1)
features.shape

torch.Size([1, 4, 256, 8, 8])

## 2. Temporal Encoder

In [ ]:
#export
class TemporalModel(nn.Module):
    def __init__(
            self, in_channels, receptive_field, input_shape, start_out_channels=64, extra_in_channels=0,
            n_spatial_layers_between_temporal_layers=0, use_pyramid_pooling=True):
        super().__init__()
        self.receptive_field = receptive_field
        n_temporal_layers = receptive_field - 1

        h, w = input_shape
        modules = []

        block_in_channels = in_channels
        block_out_channels = start_out_channels

        for _ in range(n_temporal_layers):
            if use_pyramid_pooling:
                use_pyramid_pooling = True
                pool_sizes = [(2, h, w)]
            else:
                use_pyramid_pooling = False
                pool_sizes = None
            temporal = TemporalBlock(
                block_in_channels,
                block_out_channels,
                use_pyramid_pooling=use_pyramid_pooling,
                pool_sizes=pool_sizes,
            )
            spatial = [
                Bottleneck3D(block_out_channels, block_out_channels, kernel_size=(1, 3, 3))
                for _ in range(n_spatial_layers_between_temporal_layers)
            ]
            temporal_spatial_layers = nn.Sequential(temporal, *spatial)
            modules.extend(temporal_spatial_layers)

            block_in_channels = block_out_channels
            block_out_channels += extra_in_channels

        self.out_channels = block_in_channels

        self.model = nn.Sequential(*modules)

    def forward(self, x):
        # Reshape input tensor to (batch, C, time, H, W)
        x = x.permute(0, 2, 1, 3, 4)
        x = self.model(x)
        x = x.permute(0, 2, 1, 3, 4).contiguous()
        return x[:, (self.receptive_field - 1):]

In [ ]:
tm = TemporalModel(256, 3, (8,8), 128)

In [ ]:
temp_encoded = tm(features)
temp_encoded.shape

torch.Size([1, 2, 128, 8, 8])

## 3. Future State Predictions

In [ ]:
fp = FuturePrediction(128, 128, n_gru_blocks=4, n_res_layers=4)

In [ ]:
hidden = torch.rand(1, 128, 8, 8)
x = torch.rand(1, 4, 128, 8, 8)
fp(x, hidden).shape

torch.Size([1, 4, 128, 8, 8])

## 4A. Segmentation Decoder

In [ ]:
bn = Bottleneck(256, 128, upsample=True)
bn

Bottleneck(
  (layers): Sequential(
    (conv_down_project): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (abn_down_project): Sequential(
      (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU(inplace=True)
    )
    (conv): ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1), bias=False)
    (abn): Sequential(
      (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU(inplace=True)
    )
    (conv_up_project): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (abn_up_project): Sequential(
      (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU(inplace=True)
    )
    (dropout): Dropout2d(p=0.0, inplace=False)
  )
  (projection): Sequential(
    (upsample_skip_proj): Interpolate()
    (conv_skip_proj): Conv2d(256, 128, kernel_size=(1, 1), stride=(1

In [ ]:
features[0].shape

torch.Size([4, 256, 8, 8])

In [ ]:
x = torch.rand(1,256,32,32)
bn(x).shape

torch.Size([1, 128, 64, 64])

In [ ]:
#export
class Upsampler(nn.Module):
    def __init__(self, sizes=[128,128,64], n_out=3):
        super().__init__()
        zsizes = zip(sizes[:-1], sizes[1:])
        self.convs = nn.Sequential(*[Bottleneck(si, sf, upsample=True) for si,sf in zsizes], 
                                   Bottleneck(sizes[-1], sizes[-1], upsample=True), 
                                   ConvBlock(sizes[-1], n_out, kernel_size=1, activation=None))
        
    def forward(self, x):
        return self.convs(x)

In [ ]:
us = Upsampler()

x = torch.rand(1,128,32,32)
us(x).shape

torch.Size([1, 3, 256, 256])

## 4B. Irradiance Module

In [ ]:
#export
class IrradianceModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.convs = nn.Sequential(ConvBlock(128, 64), 
                                   ConvBlock(64, 64),
                                   nn.AdaptiveMaxPool2d(1)
                                  )
        self.linear = nn.Sequential(nn.Flatten(), 
                                    nn.BatchNorm1d(64),
                                    nn.Linear(64, 1)
                                   )
    def forward(self, x):
        return self.linear(self.convs(x))

In [ ]:
im = IrradianceModule()

In [ ]:
x = torch.rand(2, 128, 32, 32)
im(x).shape

torch.Size([2, 1])

## Everything Together...

In [ ]:
#export
class Eclipse(nn.Module):
    """Not very parametric"""
    def __init__(self, n_in=3, n_out=4, horizon=5, img_size=(128, 128), debug=False):
        super().__init__()
        store_attr()
        self.spatial_downsampler = SpatialDownsampler(n_in)
        self.temporal_model = TemporalModel(256, 3, input_shape=(img_size[0]//8, img_size[1]//8), start_out_channels=128)
        self.future_prediction = FuturePrediction(128, 128, n_gru_blocks=4, n_res_layers=4)
        self.upsampler = Upsampler(n_out=n_out)
        self.irradiance = IrradianceModule()
    
    def zero_hidden(self, x, horizon):
        bs, ch, h, w = x.shape
        return x.new_zeros(bs, horizon, ch, h, w)
        
    def forward(self, imgs):
        x = torch.stack([self.spatial_downsampler(img) for img in imgs], dim=1)
        
        #encode temporal model
        states = self.temporal_model(x)
        if self.debug: print(f'{states.shape=}')
        
        #get hidden state
        present_state = states[:, -1:]
        if self.debug: print(f'{present_state.shape=}')
        
        
        # Prepare future prediction input
        hidden_state = present_state.squeeze()
        if self.debug: print(f'{hidden_state.shape=}')
        
        future_prediction_input = self.zero_hidden(hidden_state, self.horizon)
        
        # Recursively predict future states
        future_states = self.future_prediction(future_prediction_input, hidden_state)

        # Concatenate present state
        future_states = torch.cat([present_state, future_states], dim=1)
        if self.debug: print(f'{future_states.shape=}')
        
        #decode outputs
        masks, irradiances = [], []

        for state in future_states.unbind(dim=1):
            masks.append(self.upsampler(state))
            irradiances.append(self.irradiance(state))
        return {'masks': masks, 'irradiances': torch.cat(irradiances, dim=-1)}
        

In [ ]:
eclipse = Eclipse(debug=True)

In [ ]:
preds = eclipse([torch.rand(2, 3, 128, 128) for _ in range(4)])

preds['masks'][0].shape, preds['irradiances'].shape

states.shape=torch.Size([2, 2, 128, 16, 16])
present_state.shape=torch.Size([2, 1, 128, 16, 16])
hidden_state.shape=torch.Size([2, 128, 16, 16])
future_states.shape=torch.Size([2, 6, 128, 16, 16])


(torch.Size([2, 4, 128, 128]), torch.Size([2, 6]))

## Export

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_model.ipynb.
Converted 01_layers.ipynb.
Converted index.ipynb.
